#**Ch08. 텍스트분석 실습 #5**
Ch08-07. 문서 군집화 실습
* Opinion Review 데이터 셋 : https://archive.ics.uci.edu/ml/datasets/Opinosis+Opinion+%26frasl%3B+Review
* 해당 데이터 셋은 51개의 텍스트 파일로 구성돼 있으며, 각 파일은 Tripadvisor(호텔), Edmunds.com(자동차), Amazon.com(전자제품) 사이트에서 가져온 리뷰 문서임
* 각 문서는 약 100개 정도의 문장을 가지고 있음
* 원래는 토픽 모델링 논문으로 사용된 데이터 셋이나, 여기서는 문서 군집화를 이용해 각 리뷰를 분류해 볼 것임

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=False)

Mounted at /content/gdrive


In [ ]:
import zipfile
         
with zipfile.ZipFile('/content/gdrive/My Drive/Colab Notebooks/2020-TAVE-ML-Practice-Study/data/OpinosisDataset1.0.zip') as existing_zip:
  existing_zip.extractall()

**여러 개의 파일을 DataFrame으로 로딩**

1. 해당 디렉토리 내의 모든 파일에 대해 for문으로 반복하며 개별 파일명을 파일명 리스트에 추가
2. 개별 파일은 DataFrame으로 읽은 후 다시 문자열로 반환한 뒤 파일 내용 리스트에 추가
3. 만들어진 파일명 리스트와 파일 내용 리스트를 이용해 이 두개를 칼럼으로 가지는 DataFrame생성

In [ ]:
import pandas as pd
import glob, os

path = r'/content/OpinosisDataset1.0/topics'

# path의 하위 모든 .data 파일의 파일명을 리스트로 취합
all_files = glob.glob(os.path.join(path, "*.data"))
filename_list = []
opinion_text = []

# 개별 파일의 파일명은 filename_list로 취합
# 개별 파일의 파일내용은 DataFrame 로딩 후 다시 string으로 변환해 opinion_text_list로 취합
for file_ in all_files:
  # 개별 파일을 읽어서 DataFrame으로 생성
  df = pd.read_table(file_, index_col=None, header=0,  encoding='latin1')

  # 절대 경로로 주어진 파일명을 가공
  # 맨 마지막 .data 확장자도 제거
  filename_ = file_.split('/')[-1]
  filename = filename_.split('.')[0]

  # 파일명 list와 파일 내용 list에 파일명과 파일 내용을 추가
  filename_list.append(filename)
  opinion_text.append(df.to_string())

# 파일명 list와 파일 내용 list객체를 DataFrame으로 생성
document_df = pd.DataFrame({'filename':filename_list, 'opinion_text':opinion_text})
document_df.head()

,filename,opinion_text
0,speed_garmin_nuvi_255W_gps,...
1,staff_bestwestern_hotel_sfo,...
2,food_holiday_inn_london,...
3,transmission_toyota_camry_2007,...
4,video_ipod_nano_8gb,...


## TF-IDF Feature Vectorization

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [ ]:
from nltk.stem import WordNetLemmatizer
import nltk
import string

remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)
lemmar = WordNetLemmatizer()

def LemTokens(tokens):
  return [lemmar.lemmatize(token) for token in tokens]

def LemNormalize(text):
  # 소문자로 변환 후 구두점 삭제
  # 단어 토큰화 후 어근추출함수 호출
  return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vect = TfidfVectorizer(tokenizer=LemNormalize, stop_words='english', ngram_range=(1, 2), min_df=0.05, max_df=0.85)

# opinion_text 칼럼 값으로 피처 벡터화 수행
feature_vect = tfidf_vect.fit_transform(document_df['opinion_text'])

/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'le', 'u', 'wa'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


## K-means Clustering

- 문서의 유형은 크게 전자제품, 자동차, 호텔로 분류
- 각 유형은 또 세부 요소로 나뉨
(Ex. 전자제품은 네비게이션, 아이판, 랩탑 컴퓨터 등)

In [ ]:
from sklearn.cluster import KMeans

# 5개 집합으로 군집화 수행. 예제를 위해 동일한 클러스터링 결과 도출용 random_state=0
km_cluster = KMeans(n_clusters=5, max_iter=10000, random_state=0)
km_cluster.fit(feature_vect)
cluster_label = km_cluster.labels_
cluster_centers = km_cluster.cluster_centers_

In [ ]:
# 군집화 결과를 칼럼으로 추가
document_df['cluster_label'] = cluster_label
document_df.head()

,filename,opinion_text,cluster_label
0,speed_garmin_nuvi_255W_gps,...,2
1,staff_bestwestern_hotel_sfo,...,0
2,food_holiday_inn_london,...,3
3,transmission_toyota_camry_2007,...,4
4,video_ipod_nano_8gb,...,2


In [ ]:
# sort_values(by=정렬칼럼명) 수행하여 데이터 정렬
# cluster_label 별로 확인
document_df[document_df['cluster_label']==0].sort_values(by='filename')

,filename,opinion_text,cluster_label
25,free_bestwestern_hotel_sfo,...,0
1,staff_bestwestern_hotel_sfo,...,0
49,staff_swissotel_chicago,...,0


Cluster #0은 호텔에 대한 리뷰로 군집화됨

In [ ]:
document_df[document_df['cluster_label']==1].sort_values(by='filename')

,filename,opinion_text,cluster_label
36,battery-life_amazon_kindle,...,1
44,battery-life_ipod_nano_8gb,...,1
30,battery-life_netbook_1005ha,...,1
14,keyboard_netbook_1005ha,...,1
27,performance_netbook_1005ha,...,1
21,size_asus_netbook_1005ha,...,1
8,speed_windows7,...,1


Cluster #1은 킨들, 아이팟, 넷북 등의 포터블 전자기기에 대한 리뷰로 군집화됨

In [ ]:
document_df[document_df['cluster_label']==2].sort_values(by='filename')

,filename,opinion_text,cluster_label
12,accuracy_garmin_nuvi_255W_gps,...,2
42,buttons_amazon_kindle,...,2
34,directions_garmin_nuvi_255W_gps,...,2
35,display_garmin_nuvi_255W_gps,...,2
48,eyesight-issues_amazon_kindle,...,2
31,features_windows7,...,2
5,fonts_amazon_kindle,...,2
32,navigation_amazon_kindle,...,2
43,price_amazon_kindle,...,2
26,satellite_garmin_nuvi_255W_gps,...,2


Cluster #2는 Cluster #1과 비슷하게 킨들, 아이팟, 넷북이 군집에 포함돼 있지만, 주료 차량용 네비게이션으로 군집 구성

In [ ]:
document_df[document_df['cluster_label']==3].sort_values(by='filename')

,filename,opinion_text,cluster_label
9,bathroom_bestwestern_hotel_sfo,...,3
2,food_holiday_inn_london,...,3
50,food_swissotel_chicago,...,3
6,location_bestwestern_hotel_sfo,...,3
23,location_holiday_inn_london,...,3
22,parking_bestwestern_hotel_sfo,...,3
45,price_holiday_inn_london,...,3
39,room_holiday_inn_london,...,3
11,rooms_bestwestern_hotel_sfo,...,3
33,rooms_swissotel_chicago,...,3


Cluster #3은 대부분 호텔 리뷰

In [ ]:
document_df[document_df['cluster_label']==4].sort_values(by='filename')

,filename,opinion_text,cluster_label
37,comfort_honda_accord_2008,...,4
13,comfort_toyota_camry_2007,...,4
38,gas_mileage_toyota_camry_2007,...,4
29,interior_honda_accord_2008,...,4
47,interior_toyota_camry_2007,...,4
19,mileage_honda_accord_2008,...,4
17,performance_honda_accord_2008,...,4
40,quality_toyota_camry_2007,...,4
15,seats_honda_accord_2008,...,4
3,transmission_toyota_camry_2007,...,4


Cluster #4는 토요타, 혼다 등의 자동차 리뷰로 잘 군집화

In [ ]:
from sklearn.cluster import KMeans

# 3개의 집합으로 군집화 
km_cluster = KMeans(n_clusters=3, max_iter=10000, random_state=0)
km_cluster.fit(feature_vect)
cluster_label = km_cluster.labels_


# 소속 클러스터를 cluster_label 컬럼으로 할당하고 cluster_label 값으로 정렬
document_df['cluster_label'] = cluster_label
document_df.sort_values(by='cluster_label')

,filename,opinion_text,cluster_label
0,speed_garmin_nuvi_255W_gps,...,0
35,display_garmin_nuvi_255W_gps,...,0
34,directions_garmin_nuvi_255W_gps,...,0
32,navigation_amazon_kindle,...,0
31,features_windows7,...,0
30,battery-life_netbook_1005ha,...,0
28,sound_ipod_nano_8gb,headphone jack i got a clear case for it a...,0
27,performance_netbook_1005ha,...,0
41,voice_garmin_nuvi_255W_gps,...,0
42,buttons_amazon_kindle,...,0


- Cluster #0은 포터블 전자기기 리뷰로만 군집 구성
- Cluster #1은 호텔 리뷰로만 군집 구성
- Cluster #2는 자동차 리뷰로만 군집 구성

## 군집별 핵심 단어 추출하기


In [ ]:
cluster_centers = km_cluster.cluster_centers_
print('cluster_centers shape:',cluster_centers.shape)
print(cluster_centers)

cluster_centers shape: (3, 4611)
[[0.0104721  0.         0.         ... 0.00735716 0.         0.        ]
 [0.         0.00084138 0.         ... 0.         0.         0.        ]
 [0.         0.00099499 0.00174637 ... 0.         0.00183397 0.00144581]]


- (3, 4611) : 군집이 3개, word피처가 2409개로 구성
- 각 행의 배열 값은 각 군집 내의 2409개 피처 위치가 개별 중심과 얼마나 가까운가를 상대 값으로 나타낸 것
- 0~1 사이 값, 1에 가까울수록 중심과 가까운 값 의미

In [ ]:
# 군집별 top n 핵심단어, 그 단어의 중심 위치 상대값, 대상 파일명들을 반환함. 
def get_cluster_details(cluster_model, cluster_data, feature_names, clusters_num, top_n_features=10):
  # 딕셔너리 생성
  cluster_details = {}

  # cluster_centers array 의 값이 큰 순으로 정렬된 index값을 반환
  # 군집 중심점(centroid)별 할당된 word 피처들의 거리값이 큰 순으로 값을 구하기 위함.
  centroid_feature_ordered_ind = cluster_model.cluster_centers_.argsort()[:,::-1]

  # 개별 군집별로 iteration하면서 핵심단어, 그 단어의 중심 위치 상대값, 대상 파일명 입력
  for cluster_num in range(clusters_num):
    # 개별 군집별 정보를 담을 데이터 초기화.
    cluster_details[cluster_num] = {}
    cluster_details[cluster_num]['cluster'] = cluster_num

    # 중심과 가까운 피처 인덱스 이용하여 top n 피처 단어 구함.
    top_feature_indexes = centroid_feature_ordered_ind[cluster_num, :top_n_features]
    top_features = [feature_names[ind] for ind in top_feature_indexes]

    # top_feature_indexes를 이용해 해당 피처 단어의 중심 위치 상댓값 구함
    top_feature_values = cluster_model.cluster_centers_[cluster_num, top_feature_indexes].tolist()

    # cluster_details 딕셔너리 객체에 개별 군집별 핵심 단어와 중심위치 상대값, 그리고 해당 파일명 입력
    cluster_details[cluster_num]['top_features']=top_features
    cluster_details[cluster_num]['top_features_value']=top_feature_values
    filenames = cluster_data[cluster_data['cluster_label']==cluster_num]['filename']
    filenames = filenames.values.tolist()
    cluster_details[cluster_num]['filenames'] = filename_list

  return cluster_details


In [ ]:
def print_cluster_details(cluster_details):
  for cluster_num, cluster_detail in cluster_details.items():
    print('####### Cluster {0}'.format(cluster_num))
    print('Top features', cluster_detail['top_features'])
    print('Review 파일명 :', cluster_detail['filenames'][:7])
    print('==================================================')

In [ ]:
feature_names = tfidf_vect.get_feature_names()

cluster_details = get_cluster_details(cluster_model=km_cluster, cluster_data=document_df,\
                                      feature_names=feature_names, clusters_num=3, top_n_features=10)
print_cluster_details(cluster_details)

####### Cluster 0
Top features ['screen', 'battery', 'keyboard', 'battery life', 'life', 'kindle', 'video', 'direction', 'size', 'voice']
Review 파일명 : ['speed_garmin_nuvi_255W_gps', 'staff_bestwestern_hotel_sfo', 'food_holiday_inn_london', 'transmission_toyota_camry_2007', 'video_ipod_nano_8gb', 'fonts_amazon_kindle', 'location_bestwestern_hotel_sfo']
####### Cluster 1
Top features ['interior', 'seat', 'mileage', 'comfortable', 'gas', 'gas mileage', 'transmission', 'car', 'performance', 'quality']
Review 파일명 : ['speed_garmin_nuvi_255W_gps', 'staff_bestwestern_hotel_sfo', 'food_holiday_inn_london', 'transmission_toyota_camry_2007', 'video_ipod_nano_8gb', 'fonts_amazon_kindle', 'location_bestwestern_hotel_sfo']
####### Cluster 2
Top features ['room', 'hotel', 'service', 'staff', 'food', 'location', 'bathroom', 'clean', 'price', 'parking']
Review 파일명 : ['speed_garmin_nuvi_255W_gps', 'staff_bestwestern_hotel_sfo', 'food_holiday_inn_london', 'transmission_toyota_camry_2007', 'video_ipod_nan